In [ ]:
# %load_ext pycodestyle_magic
# %pycodestyle_on
# %pycodestyle_off

# void

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Import des librairies

In [ ]:
import time

import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go

import importlib

import clean_lib as cl

pass

# Fonctions

In [ ]:
def get_arity_cols(df_c1, df_c2):
    """
    Renvoie des informations sur les relations entre
    les deux colonnes de tables passées en arguments:
    - a: 1 si la colonne df_c1 est toujours renseignée
         0 sinon
    - b_min: une valeur donnée de la colonne df_c1 est
             liée au minimum à b_min valeurs différentes
             de la colonne df_c2
    - b_max: une valeur donnée de la colonne df_c1 est
             liée au maximum à b_max valeurs différentes
             de la colonne df_c2
    - b_mean: une valeur donnée de la colonne df_c1 est
              liée en moyenne à b_max valeurs différentes
              de la colonne df_c2
    """

    df = pd.DataFrame({'from': df_c1, 'to': df_c2})

    if df['from'].notna().sum() == 0:
        (a, b) = (0, 0)
    elif df.loc[df['from'].notna(), 'to'].notna().sum() == 0:
        (a, b) = (0, 0)
    else:
        if df['from'].isna().sum() > 0:
            a = 0
        else:
            a = 1

        b_min = df[df['from'].notna()].groupby('from').nunique()['to'].min()
        b_max = df[df['from'].notna()].groupby('from').nunique()['to'].max()
        b_mean = df[df['from'].notna()].groupby('from').nunique()['to'].mean()

    return(a, b_min, b_max, b_mean)


def get_arity_df(df_):
    """
    Renvoie dans un DataFrame,pour chaque paire de colonnes de df_,
    les informations fournies par la fonction get_arity_cols()
    """

    arity = pd.DataFrame(columns=['key', 'from', 'to', 'arity', 'mean'])

    for c1 in df_.columns:

        key = df_[c1].nunique() == df_.shape[0]

        for c2 in df_.columns:

            if c1 == c2:
                continue

            df_c1 = df_[c1]
            df_c2 = df_[c2]

            a, b_min, b_max, b_mean = get_arity_cols(df_c1, df_c2)

            arity_ = pd.DataFrame(
                [[key, c1, c2, (a, b_min, b_max), round(b_mean, 2)]],
                columns=['key', 'from', 'to', 'arity', 'mean']
                )
            arity = arity.append(arity_, ignore_index=True)

    return(arity)


def bizarre(df, c_from, c_to, max_to):
    """
    Extrait du DataFrame renvoyé par get_arity_df()
    les lignes pour lesquelles une valeur de la colonne c_from
    a max_to correspondances dans c_to.
    """
    group = df.groupby(c_from).nunique()[c_to]
    ind = group[group == max_to].index
    print(f"Nombre de {c_from} ayant {max_to} {c_to}: {len(ind)}")

    return(df[df[c_from].isin(ind)].sort_values(by=c_from))


def print_bizarre(df, c_from, c_to, max_to):
    """
    Idem à la fonction bizarre(), mais n'affiche que le nombre
    de lignes concernées sans les extraire. 
    """
    group = df.groupby(c_from).nunique()[c_to]
    ind = group[group == max_to].index
    print(f"Nombre de {c_from} ayant {max_to} {c_to}: {len(ind)}")

**Fonction de feature engineering**  

In [ ]:
def feat_eng(tabs, date=pd.to_datetime('01/01/2021')):
    """
    Renvoie un DataFrame dont les colonnes sont les variables construites
    à partir des données originales.
    Ne prend que les informations portant sur des commandes effectuées
    avant la date passée en argument.

    Après sélection des variables utilisées pour le clustering,
    certaines variables ont été commentées pour accélérer la fonction.
    Elles sont indiquées par (*) ci-dessous.

    Les variables contruites sont, pour chaque client identifié
    par 'customer_unique_id':
    - 'order_to_delivery': durée moyenne depuis la commande
      jusqu'à la livraison
    - (*)'order_purchasetime_stamp_max': date de la dernière commande
    - 'last_order_week': numéro de semaine de la dernière commande
    - (*)'order_purchasetime_stamp_min': date de la première commande
    - (*)'delivery_delta': écart moyen entre la date de livraison prévue
      et réelle
    - (*)'most_freq_hour': heure de commande la plus fréquente
    - (*)'most_freq_day': jour de commande le plus fréquent
    - (*)'most_freq_week': semaine de commande la plus fréquente
    - (*)'most_freq_month': mois de commande le plus fréquent
    - 'n_orders': vaut 1 si le client a passé une seule commande et 2 
       si le client a passé plusieurs commandes.
    - 'payment_value': valeur moyenne des commandes du client
    - 'review_score_mean: score moyen attribué par le client'
    """
    custs, geo, oitems, opays, orevs, \
        orders, prods, sellers, translations = tabs

    # Customers / Commandes
    custs_ = custs.copy()
    orders_ = orders.copy()
    c_ = custs_.merge(orders_, how='left', on='customer_id')

    # Filter orders prior to date
    c_ = c_.loc[c_['order_purchase_timestamp'] < date]
    orders_ = orders_.loc[orders_['order_purchase_timestamp'] < date]

    # DECOMMENTER POUR ACTIVER
    # Date "d'aujourd'hui", c'est à dire de la dernière commande réalisée
    today = orders_['order_purchase_timestamp'].max()

#     # DECOMMENTER POUR ACTIVER
#     # Nombre de commande dans chacun des order status
#     orders_status = orders_.groupby('order_status')['order_id'].count()

    # Durée du processus order to delivery (to the client)
    c_['order_to_delivery'] = \
        c_['order_delivered_customer_date'] - c_['order_purchase_timestamp']
    # Filter - Grouper - Aggréger - Selectionner - Merger - Convertir
    to_agg = c_[c_['order_to_delivery'].notna()]
    to_agg = to_agg.groupby('customer_unique_id')
    to_agg = to_agg['order_to_delivery'].mean(numeric_only=False)
    df_feat = pd.DataFrame(to_agg)
    df_feat['order_to_delivery'] = df_feat['order_to_delivery'].dt.days
    df_feat['order_to_delivery'] = df_feat.to_numpy()

    # DECOMMENTER POUR ACTIVER
    # Temps depuis la dernière commande par le client
    # Filter - Grouper - Aggréger - Selectionner - Merger - Convertir
    to_agg = c_.groupby('customer_unique_id')
    to_agg = to_agg['order_purchase_timestamp'].max()
    to_agg = today - to_agg
    df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
    columns = {'order_purchase_timestamp': 'order_purchasetime_stamp_max'}
    df_feat = df_feat.rename(columns=columns)
    feat = df_feat['order_purchasetime_stamp_max'].dt.days
    feat = feat.to_numpy()
    df_feat['order_purchasetime_stamp_max'] = feat

    # Semaine de la dernière commande par le client
    # Filter - Grouper - Aggréger - Selectionner - Merger - Convertir
    custs_ = custs.copy()
    orders_ = orders.copy()
    c_ = custs_.merge(orders_, how='left', on='customer_id')
    to_agg = c_.groupby('customer_unique_id')
    to_agg = to_agg['order_purchase_timestamp'].max().dt.week
    df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
    columns = {'order_purchase_timestamp': 'last_order_week'}
    df_feat = df_feat.rename(columns=columns)

#     # DECOMMENTER POUR ACTIVER
#     # Temps depuis la première commande par client
#     to_agg = c_.groupby('customer_unique_id')
#     to_agg = to_agg['order_purchase_timestamp'].min()
#     to_agg = today - to_agg
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
#     columns = {'order_purchase_timestamp': 'order_purchasetime_stamp_min'}
#     df_feat = df_feat.rename(columns=columns)
#     feat = df_feat['order_purchasetime_stamp_min'].dt.days
#     feat = feat.to_numpy()
#     df_feat['order_purchasetime_stamp_min'] = feat

#     # DECOMMENTER POUR ACTIVER
#     # Date de livraison - Date de livraison annoncée
#     c_['delivery_delta'] = c_['order_delivered_customer_date'] \
#         - c_['order_estimated_delivery_date']
#     to_agg = c_[c_['delivery_delta'].notna()]
#     to_agg = to_agg.groupby('customer_unique_id')
#     to_agg = to_agg['delivery_delta'].mean(numeric_only=False)
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
#     feat = df_feat['delivery_delta'] = df_feat['delivery_delta'].dt.days
#     feat = feat.to_numpy()
#     df_feat['delivery_delta'] = feat

#     # DECOMMENTER POUR ACTIVER
#     # Périodes de commande les plus fréquentes
#     def agg_mode(x):
#         return(x.mode()[0])
#     c_['most_freq_hour'] = c_['order_purchase_timestamp'].dt.hour
#     to_agg = c_.groupby('customer_unique_id')['most_freq_hour'].agg(agg_mode)
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
#     c_['most_freq_day'] = c_['order_purchase_timestamp'].dt.dayofweek
#     to_agg = c_.groupby('customer_unique_id')['most_freq_day'].agg(agg_mode)
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
#     c_['most_freq_week'] = c_['order_purchase_timestamp'].dt.week
#     to_agg = c_.groupby('customer_unique_id')['most_freq_week'].agg(agg_mode)
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
#     c_['most_freq_month'] = c_['order_purchase_timestamp'].dt.month
#     to_agg = c_.groupby('customer_unique_id')['most_freq_month']
#     to_agg = to_agg.agg(agg_mode)
#     df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')

    # Nombre de commandes par client au total
    to_agg = c_.groupby('customer_unique_id')['order_id'].count()
    df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
    df_feat = df_feat.rename(columns={'order_id': 'n_orders'})
    # Si supérieur ou égal à 2 commandes, indiquer 2
    df_feat.loc[df_feat['n_orders'] >= 2, 'n_orders'] = 2

    # Price and Payments
    to_agg = oitems.groupby('order_id')['price'].sum()
    c_ = c_.merge(to_agg, how='left', on='order_id')
    to_agg = oitems.groupby('order_id')['freight_value'].sum()
    c_ = c_.merge(to_agg, how='left', on='order_id')
    c_['payment_value_1'] = c_['price'] + c_['freight_value']
    to_agg = opays.groupby('order_id')['payment_value'].sum()
    c_ = c_.merge(to_agg, how='left', on='order_id')
    c_['delta_price_pay'] = round((c_['payment_value_1']
                                   - c_['payment_value']))

    # Merge df_clustering
    to_agg = c_.groupby('customer_unique_id')['payment_value'].mean()
    df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')
    # NB : des clients ont un solde non défini quand la commande n'a pas abouti
    # On remplace les np.nan par zero
    ind = df_feat.loc[:, 'payment_value'].isna()
    df_feat.loc[ind, 'payment_value'] = 0

    # Review score
    to_agg = orevs.groupby('order_id')['review_score'].mean()
    c_ = c_.merge(to_agg, how='left', on='order_id')
    c_ = c_.rename(columns={'review_score': 'review_score_mean'})
    to_agg = c_.groupby('customer_unique_id')['review_score_mean'].mean()
    df_feat = df_feat.merge(to_agg, how='left', on='customer_unique_id')

    return(df_feat)

**Fonction de preprocessing**

In [ ]:
def preprocess(df):
    """
    Fonction de normalisation et centrage.
    """

    from sklearn.preprocessing import StandardScaler

    df_preprocess = df.copy()
    sc = StandardScaler()
    df_preprocess[:] = sc.fit_transform(df_preprocess)

    return(df_preprocess)

**Fonction de clustering - KMeans**

In [ ]:
def compute_KMeans(df, cols, clusters_range, file_name, simu=False):
    """
    Si simu est True:
        Calcule le K-Means et le score de silhouette sur le DataFrame df
        pour différents k pris dans clusters_range. Sauvegarde les scores
        de silhouette dans le fichier file_name et renvoie la liste de ces
        scores.

    Sinon:
        Charge les scores de silhouette depuis le fichier file_name.
        Renvoie la liste des scores de silhouette.
    """

    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score

    if simu:
        silhouette = []
        for i, n_clusters in enumerate(cluster_range):
            print(f"Simu {i+1} / {len(cluster_range)} en cours")
            km = KMeans(n_clusters=n_clusters)
            km.fit(df.loc[:, cols])
            silhouette.append(silhouette_score(df.loc[:, cols], km.labels_))

        with open(file_name, 'wb') as f:
            pickle.dump(silhouette, f, protocol=pickle.HIGHEST_PROTOCOL)
            print(f"\nTables dumpées dans un fichier pickle")
    else:
        silhouette = pickle.load(open(file_name, "rb"))
        print(f"\nSilhouettes importées depuis le fichier pickle")

    return(silhouette)

**Fonction de clustering - DBSCAN - Scan des densités du jeu de données**

In [ ]:
def compute_density(tabs, cols, eps, compute_db_params=False):
    """
    Entrées:
    tabs: la liste des tables de données
    cols: les variables issues du features engineering
        auxquelles on s'intéresse
    eps: la liste des epsilons caractérisant le rayon d'une sphère autour
        d'un point dans laquelle on va rechercher le nombre de voisins
    compute_db_params: calculer les résultats si True et
        les charger depuis un fichier sinon.

    Si compute_db_params est True:
        - Construit les variables du feature enginering dans df
        - Sélectionne les variables df[cols]
        - Centre et normalise df[cols]
        - Calcule pour chaque point
            1) La distance au plus proche voisin
            2) Pour chaque epsilon de eps, le nombre de voisins du point
               dans la sphère de rayon epsilon
        - Sauvegarde les résultats dans un fichier
        - Renvoie les résultats sous forme de DataFrame

    Sinon:
        - Charge les résultats à partir d'un fichier
        - Renvoie les résultats sous forme de DataFrame
    """

    import numpy.linalg as LA
    if compute_db_params:

        # Construction des variables
        df = feat_eng(tabs, date='06/11/2020')

        # Nombre d'individus
        n = len(df)

        # Selection des variables
        df = df[cols]

        # Preprocessing et mise en forme
        df = preprocess(df)
        df = np.array(df)

        # Calcul de la distance au plus proche voisin pour chaque individu
        db_cols = ['dist_min']
        db_cols.extend([str(round(e, 1)) for e in eps])
        db = pd.DataFrame(columns=db_cols)

        for i in range(len(df)):
            if i % 10000 == 0:
                print(i)
            v = df[i, :].reshape(1, -1)
            norm = LA.norm(df - v, axis=1)
            norm = np.delete(norm, i)
            norm = np.sqrt(norm)
            out = [norm.min()]
            out.extend([(norm < eps).sum() for eps in eps])
            db.loc[i] = out

        with open('db.p', 'wb') as f:
            pickle.dump(db, f, protocol=pickle.HIGHEST_PROTOCOL)
            print(f"\nParamètres DB dumpées dans un fichier pickle")

    else:
        db = pickle.load(open('db.p', "rb"))
        print(f"\nParamètres DB importés depuis le fichier pickle")

    return(db)

# Paramétrage du notebook

In [ ]:
# Faut-il importer les données d'origines (True)
# ou les données retravaillées (False)
# Les données retravaillées contiennent les informations
# sur les relations entre colonnes des tables
do_import = False

# Faut-il explorer les relations entre colonnes des tables ?
explore_arity = True

# Faut-il explorer les variables des tables d'origine ?
explore_tables = True

# Faut-il explorer les variables construites ?
explore_features = True

# Faut-il calculer les informations de densité ?
compute_db_params = False

# Paramètres d'affichage
pd.set_option('max_r', 20)
pd.options.display.max_rows = 999

# Import des données

In [ ]:
# If do_import is True, datasets are uploaded and stored in a dictionnary
if do_import:
    custs = pd.read_csv("Data/olist_customers_dataset.csv")
    geo = pd.read_csv("Data/olist_geolocation_dataset.csv")
    oitems = pd.read_csv("Data/olist_order_items_dataset.csv",
                         parse_dates=[4],
                         infer_datetime_format=True)

# If do_import is True, compute the relationships between tables' columns
if do_import:
    print(f"\nDébut du calcul des arités")
    for t in tables:
        print(f"Calcul de l'arité de {t}")
        tables[t]['arity'] = get_arity_df(tables[t]['df'])
    print(f"Fin du calcul des arités")

    # Store tables in a pickle file
    with open('tables.p', 'wb') as f:
        pickle.dump(tables, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"\nTables dumpées dans un fichier pickle")

# If do_import is False, upload the tables from the pickel file
if not do_import:
    tables = pickle.load(open("tables.p", "rb"))
    print(f"\nTables importées depuis le fichier pickle")

    # Raccourcis vers les tables
    custs = tables['customers']['df']
    geo = tables['geographic location']['df']
    oitems = tables['order items']['df']
    opays = tables['order payments']['df']
    orevs = tables['order reviews']['df']
    orders = tables['orders']['df']
    prods = tables['products']['df']
    sellers = tables['sellers']['df']
    translations = tables['translations']['df']

tabs = [custs, geo, oitems, opays, orevs, orders, prods, sellers, translations]

# Exploration des tables

## Exploration des tables: liaisons entre colonnes

**Exemple d'exploration des liaisons entre colonnes** 

Ci-dessous, on montre un exemple de calcul de relations entre colonnes pour la table de clients.  

Les colonnes ont la signification suivante:  
- 'key': est-ce que la variable 'from' est une clé unique de la table ?  
- 'arity' = (p,min,max)
    - p: la variable 'from' est-elle toujours présente (1:True, 0: False)
    - min: la variable 'from' est liée à au moins min variables 'to
    - max: la variable 'from' est liée à au plus max variable 'to'
    - 'mean': la variable 'from' est liée en moyenne à 'mean' variable 'to'

In [ ]:
if explore_arity:
    t = tables['customers']['arity']
else:
    t = None
t

**Bizarre - Exemple**  
On voit par exemple à la ligne 10 qu'un 'customer_zip_code_prefix' peut avoir jusqu'à 3 correspondances 'customer_city'. On regarde alors avec la fonction bizarre() quels sont ces cas de figure. Il s'agit d'un cas isolé lié à une erreur de frappe et d'entrée.

In [ ]:
if explore_arity:
    b = bizarre(tables['customers']['df'],
                'customer_zip_code_prefix', 'customer_city', 3)
else:
    b = None
b

**Points remarquables issus de l'exploration des liaisons entre colonnes**  
Ci-dessous, les points remarquables issus de l'exploration des relations entre colonnes

In [ ]:
if explore_arity:
    print("\nCUSTOMERS")
    print(("Un client unique peut être lié à plusieurs client_id "
           "car le client_id identifie en fait une commande"))
    print("client_id est en fait redondant avec order_id")
    df = tables['customers']['df']
    df = df.merge(tables['orders']['df'], how='outer', on='customer_id')
    print(f"Nombre de customer_id sans order_id correspondant: \
          {df['order_id'].isna().sum()}")
    print(f"Nombre de order_id sans customer_id correspondant: \
          {df['customer_id'].isna().sum()}")

    print("\nORDER ITEMS")
    print(("Une commande contient plusieurs lignes de commande, "
           "les order items"))
    print("La clé de la table est order_id + order_item_id")
    print(("Chaque ligne de commande est décrite par un produit, "
           "vendeur, une date limite d'expédition, un prix et des frais de port"))

    print("\nPAIMENTS")
    print(("Un client peut payer par plusieurs moyens, "
           "dans ce cas une séquence de paiement est créée"))
    print(("Cela explique qu'une commande puisse "
           "être associée à plusieurs paiements"))
    print(("Chaque paiement peut être payée "
           "en plusieurs versements ..."))
    print(("... OU chaque paiment doit être fait "
           "nombre de versements fois [A confirmer]"))
    print_bizarre(tables['order payments']['df'],
                  'order_id', 'payment_value', 25)

    print("\nREVUES")
    print("Une commande peut-être liée à plusieurs revues, pas choquant")
    print_bizarre(tables['order reviews']['df'], 'order_id', 'review_id', 1)
    print_bizarre(tables['order reviews']['df'], 'order_id', 'review_id', 2)
    print_bizarre(tables['order reviews']['df'], 'order_id', 'review_id', 3)
    print_bizarre(tables['order reviews']['df'], 'order_id', 'review_id', 4)

    print("\nUne revue peut-être liée à plusieurs commandes, plus surprenant")
    print("Ca semble être des revues identiques sur un groupe de commandes")
    print_bizarre(tables['order reviews']['df'], 'review_id', 'order_id', 1)
    print_bizarre(tables['order reviews']['df'], 'review_id', 'order_id', 2)
    print_bizarre(tables['order reviews']['df'], 'review_id', 'order_id', 3)
    print_bizarre(tables['order reviews']['df'], 'review_id', 'order_id', 4)

## Exploration des tables: valeurs uniques, taux de remplissage, type de données

On voit ci-dessous que les données sont propres. Il n'y a pas de valeurs manquantes sauf pour certaines commandes en cours, des revues sans titre ou sans commentaire et un groupe de 610 produits.  
Cela n'affecte pas le clustering ultérieur.

In [ ]:
if explore_tables:
    importlib.reload(cl)

    for t in tables:
        print("\nTable: ", t)
        var_info = cl.get_var_info(tables[t]['df'],
                                   how_few=0,
                                   what='variables')
        tables[t]['variables'] = var_info
        display(HTML(tables[t]['variables'].to_html()))

## Exploration des tables: statistiques sur les colonnes

**Distribution des dates de commande, dates de scoring et montants payés**  
Ci-dessous les distributions des variables qui nous servent à construire les variables sur lesquelles on fait le clustering.  
En particulier, on remarque que les données s'arrêtent en réalité fin août.

In [ ]:
fig = px.histogram(tables['orders']['df'], x='order_purchase_timestamp', title='Distribution des dates de commande', nbins = 180)
fig.show('notebook')

In [ ]:
fig = px.histogram(tables['order reviews']['df'], x='review_creation_date', title='Distribution des dates de scoring')
fig.show('notebook')

In [ ]:
fig = px.histogram(tables['order payments']['df'], x='payment_value', title='Distribution des montants payés')
fig.show('notebook')

## Exploration des tables: doublons

Il n'y a pas de doublons dans les tables sauf dans celle des coordonnées géographiques qu'on n'utilise pas.

In [ ]:
if explore_tables:
    for t in tables:
        print(f"Doublons dans la table {t}: \
        {tables[t]['df'].duplicated(keep=False).sum()}")

# Feature engineering

**On crée la table sur laquelle on va appliquer le clustering grace à la fonction feat_eng() définie plus haut**

Rappel des variables crées:

    """
    Les variables contruites sont, pour chaque client identifié
    par 'customer_unique_id':
    - 'order_to_delivery': durée moyenne depuis la commande
      jusqu'à la livraison
    - (*)'order_purchasetime_stamp_max': date de la dernière commande
    - 'last_order_week': numéro de semaine de la dernière commande
    - (*)'order_purchasetime_stamp_min': date de la première commande
    - (*)'delivery_delta': écart moyen entre la date de livraison prévue
      et réelle
    - (*)'most_freq_hour': heure de commande la plus fréquente
    - (*)'most_freq_day': jour de commande le plus fréquent
    - (*)'most_freq_week': semaine de commande la plus fréquente
    - (*)'most_freq_month': mois de commande le plus fréquent
    - 'n_orders': vaut 1 si le client a passé une seule commande et 2 
       si le client a passé plusieurs commandes.
    - 'payment_value': valeur moyenne des commande du client
    - 'review_score_mean: score moyen attribué par le client'
    """

In [ ]:
df_clustering = feat_eng(tabs)

Ci-dessous, la table créée pour le clustering qui ne contient que les valeurs qu'on a retenu au final pour le clustering.  
Les autres variables construites dans le feature engineering ont été by-passées pour accélérer la fonction.

In [ ]:
df_clustering.head()

**Exploration des données du features engineering**

Taux de remplissage de la table

In [ ]:
if explore_features:
    importlib.reload(cl)
    var_info = cl.get_var_info(df_clustering, how_few=0, what='variables')
var_info

Statistiques des variables

In [ ]:
if explore_features:
    importlib.reload(cl)
    var_info = cl.get_var_info(df_clustering, how_few=0, what='stats')
var_info

Distribution des variables

In [ ]:
if explore_features:
    for c in df_clustering.columns:
        fig = px.histogram(df_clustering, x=c)
        fig.show('notebook')

# K-Means

**Calcul du score de silhouette de K-Means appliqué à différents jeux de variables**  

Ci-dessous on calcule pour différents k et différents jeux de variables construites les scores de silhouette des clusterings obtenus par K-Means.  

Les graphs 'silhouette 1 à 4' plus bas correspondent à  des clustering effectués sur différents jeux de variables.

On a choisit des variables pour avoir des informations de temps, fréquence d'achat, montant payé et satisfaction.

On a testé d'abord la configuration suivante:  
1) 'order_purchasetime_stamp_max', 'n_orders', 'payment_value', 'review_score_mean'  

Puis on a testé la configuration ci-dessous ou on a remplacé 'order_purchasetime_stamp_max' par 'most_freq_week':   
2) 'most_freq_week', 'n_orders', 'payment_value', 'review_score_mean'  

Puis on a recommencé en rajoutant aux configuration ci-dessus la variable 'delivery_delta':  
3) 'order_purchasetime_stamp_max', 'n_orders', 'payment_value', 'review_score_mean', 'delivery_delta'  
4) 'most_freq_week', 'n_orders', 'payment_value', 'review_score_mean', 'delivery_delta'  

In [ ]:
# Init
cluster_range = range(2, 10)
do_simu = False

# Construction des variables
df = feat_eng(tabs, date='06/11/2020')

# Preprocessing et mise en forme
df = preprocess(df)

cols = ['order_purchasetime_stamp_max',
        'n_orders',
        'payment_value',
        'review_score_mean', ]
silhouette1 = compute_KMeans(df, cols, cluster_range,
                             'kmeans1.p', simu=do_simu)

cols = ['most_freq_week',
        'n_orders',
        'payment_value',
        'review_score_mean', ]
silhouette2 = compute_KMeans(df, cols, cluster_range,
                             'kmeans2.p', simu=do_simu)

cols = ['order_purchasetime_stamp_max',
        'n_orders', 'payment_value',
        'review_score_mean',
        'delivery_delta']
silhouette3 = compute_KMeans(df, cols, cluster_range,
                             'kmeans3.p', simu=do_simu)

cols = ['most_freq_week',
        'n_orders',
        'payment_value',
        'review_score_mean',
        'delivery_delta']
silhouette4 = compute_KMeans(df, cols, cluster_range,
                             'kmeans4.p', simu=do_simu)

silhouettes = np.array([silhouette1, silhouette2, silhouette3, silhouette4])
columns = ['silhouette1', 'silhouette2', 'silhouette3', 'silhouette4']
silhouettes = pd.DataFrame(silhouettes.T,
                           columns=columns,
                           index=cluster_range)

**Graph des silhouettes**

Explication des configuration silhouette 1 à 4 à la cellule précédente.  

**Les configurations 1 et 2 sont meilleures.**  
**Le meilleur score est atteint pour 5 ou 6 clusters, on choisit de prendre 5 clusters pour faciliter l'interprétation**  

La configuration 2 donne un score encore meilleur pour 2 clusters en séparant très bien les clients qui achètent plusieurs fois des autres, mais cette segmentation à 2 classes est trop pauvre et on l'écarte.

In [ ]:
title = 'K-Means: Silouhettes en fonction de k pour différentes configurations'
labels = {'index': 'Nombre de clusters',
          'value': 'Score de silhouette'}
px.line(silhouettes,
        title=title,
       labels = labels)

**Représentation des classes du K-Means clustering pour la configuration 2 et 5 classes**

In [ ]:
from sklearn.cluster import KMeans

# Init
cols = ['last_order_week', 'n_orders', 'review_score_mean', 'payment_value', ]

# Construction des variables
df = feat_eng(tabs, date='06/11/2020')

# Preprocessing et mise en forme
df_ = preprocess(df)

km = KMeans(n_clusters=5)
km.fit(df_.loc[:, cols])
df['label'] = km.labels_

In [ ]:
# Nombre de clients par cluster
title = 'Histogramme de la population de clients par classe'
px.histogram(df, x='label',
             title=title,
            labels=labels)

**Interprétation des classes**  
Pour interpréter les classes on représente ci-dessous la distribution des variables en fonction des classes.  

La majorité des clients sont des clients satisfaits qui font un seul achat de faible valeur en début ou en fin d’année.  
Certains clients se distinguent par le fait qu’ils:  
- commandent plusieurs fois
- ou, sont insatisfaits
- ou, ont un panier moyen plus élevé
  
Attention: il y a 15 000 clients insatisfaits ! Les clients qui paient plus ou achètent plus souvent sont aussi plus exigeants.  

In [ ]:
for var in [c for c in cols if c != 'label']:
    fig = go.Figure()
    fig.add_trace(go.Box(x=df['label'], y=df[var], boxmean=True))
    fig.update_layout(title=str(var))
    fig.show('notebook')

**Calcul de la dérive dans le temps du modèle de K-Means clustering**

Dans le code ci-dessous, on mesure comment un modèle fourni à une date donnée à un client dérive au cours du temps.  
La mesure utilisée est le 'Adjusted Rand Index'.  
A travers ce score, on compare le modèle fourni à des modèles recalculés à différentes dates postérieures à la date de fourniture du modèle.

In [ ]:
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Init
n_clusters = 5
load_from_pickel = True
cols = ['last_order_week', 'n_orders', 'review_score_mean', 'payment_value', ]

start_dates = pd.date_range('31/12/2016', periods=7, freq='3M')
simus = {}

if not load_from_pickel:
    for start_date in start_dates:

        periods = pd.date_range(start_date, periods=24, freq='W')
        print(f"\nModèle livré le = {periods[0]}")

        # Modèle de référence pour la période commençant à periods[0]
        df = feat_eng(tabs, date=periods[0])
        df = df[cols]
        sc = StandardScaler()
        df[:] = sc.fit_transform(df)
        km = KMeans(n_clusters=n_clusters)
        km.fit(df)

        x = []
        y = []
        means = []
        stds = []

        for date in periods:

            if date > pd.to_datetime('2018-10-01'):
                continue

            df_ = feat_eng(tabs, date=date)
            df_ = df_[cols]
            
#             means.append(df_.mean(axis=0))
#             stds.append(df_.std(axis=0))
            
            # Modèle recalculé pour comparer au modèle de référence
            # et appliqué aux données
            df_true = df_.copy()
            sc_true = StandardScaler()
            df_true[:] = sc_true.fit_transform(df_true)
            km_true = KMeans(n_clusters=n_clusters)
            km_true.fit(df_true)
            labels_true = km_true.labels_

            # Modèle de référence appliqué aux données
            df_predict = df_.copy()
            df_predict[:] = sc.transform(df_predict)
            labels_predict = km.predict(df_predict)
            
#             # Calcul de la dérive des centres de clusters
#             centers_true = km_true.cluster_centers_
#             centers_predict = km.cluster_centers_
            
#             dist = np.zeros((n_clusters, n_clusters))
#             for i in range(n_clusters):
#                 d = centers_true - (centers_predict[i,:].reshape(1,centers_predict.shape[1]))
#                 d = d**2
#                 d = d.sum(axis=1)
#                 dist[:,i] = np.sqrt(d)
                
            ari = adjusted_rand_score(labels_true, labels_predict)
    
#             print(np.round(dist,2))
            print(f"Score le = {date}: {ari}")
    
            if ari < 0.8:
                break

            x.append(date)
            y.append(ari)
        
        simus[str(start_date)] = [x, y, means, stds]

    with open('derive.p', 'wb') as f:
        pickle.dump(simus, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"\nDérives dumpées dans un fichier pickle")
else:
    simus = pickle.load(open('derive.p', "rb"))
    print(f"\nDérives importées depuis le fichier pickle")

**Graph de la dérive dans le temps du modèle de K-Means clustering**

In [ ]:
columns = ('seuil',
           'Mise à disposition du modèle',
           'Robustesse en nombre de semaines')

derive = pd.DataFrame(columns=columns)

for seuil in [0.95, 0.90, 0.85]:
    s = []
    x = []
    y = []
    for start in simus:
        x.append(start)
        sup_seuil = np.append((np.array(simus[start][1]) > seuil), False)
        y.append(np.argmin(sup_seuil))
        s.append(seuil)
    derive_tmp = np.array([s, x, y]).T
    derive_tmp = pd.DataFrame(derive_tmp, columns=columns)
    derive = pd.concat([derive, derive_tmp])

In [ ]:
px.bar(derive,
       x='Mise à disposition du modèle',
       y='Robustesse en nombre de semaines',
       color='seuil',
       barmode='group',
       title="Robustesse du modèle K-Means")

**Le modèle de référence reste de bonne qualité (score entre 0.8 et 0.9) si il est remis à jour toutes les 3 semaines**  
Au démarrage, la robustesse n'est que de deux semaines car le nombre de clients est faible et non représentatif de la population ultérieure.

# DBSCAN  
On teste ici si l'algorithme DBScan, basé sur l'exploitation de la densité du jeu de données est pertinent. On va constater que DBScan n'est pas pertinent pour le jeu de données car les points sont répartis dans l'espace selon des densités non homogènes.

**Pour paramétrer l'algorithme, on calcule d'abord des informations de densité sur le jeu de données:**  
Pour chaque point, (c'est à dire chaque client):
- La distance au plus proche voisin
- Le nombre de voisins à une distance inférieure à r pour r dans 0.1 - 2.0

In [ ]:
eps = np.linspace(0.1, 2, 20)
cols = ['last_order_week', 'n_orders', 'review_score_mean', 'payment_value', ]

db = compute_density(tabs, cols, eps, compute_db_params=False)
db

**Choix de la résolution du clustering**  
Pour trouver un epsilon cohérent de la densité du jeu de données, on trace la courbe de la distance au plus proche voisin de chaque point. 
(Les points sont classés par distance croissante)

In [ ]:
db_min = db['dist_min'].sort_values()
title = 'Distance au plus proche voisin de chaque client'
labels = {'index': 'Client', 'value': 'Distance au plus proche voisin'}
px.line(db_min.to_numpy(),
        title=title,
        labels=labels)

Le coude de la courbe indique la zonne dans laquelle choisir un epsilon cohérent de la densité de jeu de données.  
Ici: **on choisit 0.3** qui permet d'avoir suffisamment de clusters (pour des epsilons plus grands, les clusters sont agglomérés les uns aux autres (en d'autres termes, on perd trop en résolution)

**Choix de la densité minimale de clustering**  
Pour le epsilon retenu, on trace la courbe du nombre de voisins à moins de r = 0.3 de chaque client  
C'est une aide au choix de min_samples.

In [ ]:
db_min = db['0.3'].sort_values(ascending=False)
title = 'Nombres de voisins à moins de r = 0.3 de chaque client'
labels = {'index': 'Client', 'value': 'Nombres de voisins à moins de r = 0.3'}
px.line(db_min.to_numpy(), title=title, labels = labels)

Par essai-erreurs on choisit min_samples de plus en plus grand pour diminuer le nombre de clusters obtenus en surveillant le nombre de clients considérés comme du bruit par l'algorithme.   
Avec epsilon = 0.3, **on choisit min_samples = 80** pour obtenir 6 clusters en plus des points considérés comme du 'bruit'

In [ ]:
from sklearn.cluster import DBSCAN

# Init
cols = ['last_order_week', 'n_orders', 'review_score_mean', 'payment_value', ]

# Construction des variables
df = feat_eng(tabs, date='06/11/2020')
df = df[cols]

# Preprocessing
df_ = preprocess(df)

In [ ]:
# DBSCAN
df_ = np.array(df_)
eps = 0.3
min_samples = 80
dbs = DBSCAN(eps=eps, min_samples=min_samples, )
dbs.fit(df_)
print(f"\n: Simu: eps = {eps}, min_samples = {min_samples}")
print(f"Noisy points: {(dbs.labels_ == -1).sum()}")
print(f"Clusters: {dbs.labels_.max() + 1}")
df['labels'] = dbs.labels_
clusters_size = [(df['labels'] == i).sum()
                 for i in np.sort(df['labels'].unique())]
print(f"Taille des clusters: \
    {list(zip(np.sort(df['labels'].unique()),clusters_size))}")

**Représentation des classes de DBScan clustering pour epsilon = 0.3 et min_samples = 80**

In [ ]:
# Nombre de clients par cluster
px.histogram(df, x='labels', title = 'Histogramme de la population de clients par classe')

**Interprétation des classes**  
Pour interpréter les classes on représente ci-dessous la distribution des variables en fonction des classes.  

On remarque que sur les 6 classes identifiées par DBScan, 5 correspondent au score moyen attribué au client. Le score moyen découpe l'espace en hyperplans bien séparés les uns des autres (il n'y a que 5 notes possibles). Sur ces plans les autres variables qui ont plus de modalités sont plus denses et forment les clusters.  

De la même façon, DBScan identifie un plan bien séparé concernant la variable qui indique si un client a acheté une fois ou plusieurs fois (2 modalités uniquement).  

Par ailleurs les clients qui paient plus que les autres sont trop éloignés des autres et sont considérés comme du bruit par l'algorithme.

**En raison de l'hétérogénéité de la densité des points, l'algorithme n'est pas adapté au problème**

In [ ]:
for var in [c for c in cols]:
    fig = go.Figure()
    fig.add_trace(go.Box(x=df['labels'], y=df[var], boxmean=True))
    fig.update_layout(title=str(var))
    fig.show('notebook')

# Clustering hiérarchique

On teste ici si l'algorithme de classification ascendante hiérarchique est pertinent. On va constater qu'avec la métrique adéquate, il produit des classes similaires à celle de K-Means. Toutefois, on lui préferera le K-Means qui est plus rapide.

**On applique l'algorithme à un sous-échantillon du jeu de données**

In [ ]:
from sklearn.utils.random import sample_without_replacement
from sklearn.cluster import AgglomerativeClustering

# Init
cols = ['last_order_week', 'n_orders', 'review_score_mean', 'payment_value', ]

# Construction des variables
df = feat_eng(tabs, date='06/11/2020')
df = df[cols]

# Sampling
random_state = 47
n_population = len(df)
n_sample = round(n_population * 0.4)
samples = sample_without_replacement(n_population,
                                     n_sample,
                                     random_state=random_state)
df = df.iloc[samples]

# Preprocessing
df_ = preprocess(df)

In [ ]:
# CAH
df_ = np.array(df_)
agglo = AgglomerativeClustering(distance_threshold=None,
                                n_clusters=5,
                                linkage='ward')
agglo.fit(df_)

**Représentation des classes de la CAH  pour n_clusters = 5**

In [ ]:
df['labels'] = agglo.labels_

# Nombre de clients par cluster
px.histogram(df, x='labels', title = 'Histogramme de la population de clients par classe')

**Interprétation des classes**  
Ci-dessous, on constate qu'on retrouve des classes similaires à celle du clustering K-Means

In [ ]:
df['labels'] = agglo.labels_
for var in [c for c in cols]:
    fig = go.Figure()
    fig.add_trace(go.Box(x=df['labels'],
                         y=df[var],
                         boxmean=True))
    fig.update_layout(title=str(var))
    fig.show('notebook')

# Annexes

In [ ]:
# #Rand Index calculation

# # a: groupés dans true and predict
# # d: séparés dans true et predict
# # b: séparés dand true mais groupés dans predict
# # c: groupés dans true mais séparés dans predict

# a = float(0)
# d = float(0)
# b = float(0)
# c = float(0)
# n = len(labels_predict)

# for i in range(n):
#     grouped_true = (labels_true == labels_true[i])
#     grouped_predict = (labels_predict == labels_predict[i])
#     a = a + (grouped_true &  grouped_predict).sum() - 1
#     d = d + ((~ grouped_true) & (~ grouped_predict)).sum()
#     b = b + (grouped_true & (~ grouped_predict)).sum()
#     c = c + ((~grouped_true) & grouped_predict).sum()

# rand_index = (a+b) / (n*(n-1))